# Generating Comprehensive Content and Long-Form Output

This notebook explores how to use Claude 3.7 Sonnet's extended thinking capability to generate high-quality long-form content. You'll learn:

1. How to generate comprehensive business reports
2. Techniques for creative writing and storytelling
3. Approaches for technical documentation
4. Best practices for managing and validating long outputs

## Prerequisites

- Basic understanding of Claude 3.7's extended thinking capability (covered in previous lessons)
- Familiarity with the Bedrock API and our utility functions
- Python programming experience

Let's begin by setting up our environment and importing the necessary libraries.

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Import our utility functions from previous lessons
import claude_utils

# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Claude 3.7 Sonnet model ID
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)

## Understanding Long-Form Content Generation

Before diving into specific content types, it's important to understand how Claude 3.7's extended thinking capability enhances long-form content generation:

1. **Structured Planning**: Claude can develop detailed outlines before generating content
2. **Consistent Context**: Extended thinking helps maintain consistency across longer pieces
3. **Iterative Development**: Content can be developed in stages with progressive refinement
4. **Content Validation**: Built-in mechanisms help ensure quality and completeness

Let's start with a simple example of generating a structured business report.

### Note on Utility Functions

This lesson uses several helper functions from our `claude_utils.py` module for analyzing and validating content:

- `analyze_technical_doc`: Analyzes structure and components of technical documentation
- `analyze_creative_writing`: Analyzes structure and elements of creative writing
- `validate_long_form_content`: Performs basic validation of content structure and completeness

These utilities handle the "behind the scenes" work of content analysis, allowing us to focus on the core concepts of generating different types of long-form content with Claude 3.7's extended thinking capability.

If you're interested in the implementation details of these utility functions, you can find them in the `claude_utils.py` file. However, understanding their implementation isn't necessary for learning the concepts in this lesson.

Now, let's begin exploring how to generate comprehensive content...


## 1. Generating Business Reports

Let's start by creating a comprehensive business report. We'll use a real-world scenario that requires analysis, data presentation, and recommendations.

First, we'll create a helper function that implements our best practices for report generation:
1. Start with a clear outline
2. Use structured sections
3. Include data analysis
4. Provide actionable recommendations

In [ ]:
def generate_report(prompt, reasoning_budget=4096, max_tokens=2000):
    """
    Generate a comprehensive business report using Claude's extended thinking
    
    Args:
        prompt (str): The report topic and requirements
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        str: The generated report
    """
    # Create a structured prompt that encourages good report format
    structured_prompt = f"""
    Please create a comprehensive business report on the following topic:
    
    {prompt}
    
    Follow this structure:
    1. Executive Summary
    2. Background and Context
    3. Analysis and Findings
    4. Recommendations
    5. Implementation Plan
    6. Conclusion
    
    Make each section detailed and thorough. Support your analysis with clear reasoning.
    Format the report using markdown for better readability.
    """
    
    # Invoke Claude with extended thinking
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        structured_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=max_tokens
    )
    
    return claude_utils.extract_response_content(response), response

### Testing Report Generation

Let's test our report generation function with a realistic business scenario: analyzing the impact of remote work on employee productivity and company culture.

This example will demonstrate how Claude can:
- Develop a structured analysis
- Consider multiple factors
- Provide data-driven insights
- Make practical recommendations

In [ ]:
# Define our report topic
report_topic = """
Analyze the impact of remote work on employee productivity and company culture at a mid-sized technology company (500 employees).
Consider:
- Productivity metrics before and after remote work adoption
- Employee satisfaction and retention
- Communication and collaboration effectiveness
- Impact on hiring and recruitment
- Cost implications (office space, utilities, equipment)

Provide specific recommendations for optimizing the hybrid work model going forward.
"""

# Generate the report
report_content, report_response = generate_report(
    report_topic,
    reasoning_budget=6144,  # Larger budget for detailed analysis
    max_tokens=2500        # Allow for longer output
)

# Display the report with metrics
print("Report Generation Metrics:")
print(f"Time taken: {report_response.get('_elapsed_time', 0):.2f} seconds")
print(f"Input tokens: {report_response.get('usage', {}).get('inputTokens', 0)}")
print(f"Output tokens: {report_response.get('usage', {}).get('outputTokens', 0)}\n")

# Display the report content
display(Markdown(report_content))

### Analyzing Report Quality

Now that we've generated our first report, let's examine what makes it effective and how we can improve it further. Key aspects to consider:

1. **Structure**: How well-organized is the content?
2. **Depth**: Are topics explored thoroughly?
3. **Supporting Evidence**: How well are claims supported?
4. **Actionability**: How practical are the recommendations?

Let's create a function to help evaluate and improve report quality.

In [ ]:
def analyze_report_quality(report_content):
    """
    Analyze the quality of a generated report
    
    Args:
        report_content (str): The report content to analyze
        
    Returns:
        dict: Quality metrics and suggestions for improvement
    """
    # Split the report into sections
    sections = report_content.split('\n#')
    
    # Basic metrics
    metrics = {
        "section_count": len(sections),
        "total_length": len(report_content),
        "sections": {}
    }
    
    # Analyze each section
    for section in sections:
        if section.strip():
            # Get section title (first line)
            title = section.split('\n')[0].strip('# ')
            content = '\n'.join(section.split('\n')[1:])
            
            metrics["sections"][title] = {
                "length": len(content),
                "paragraphs": len([p for p in content.split('\n\n') if p.strip()]),
                "bullet_points": len([l for l in content.split('\n') if l.strip().startswith('-')])
            }
    
    return metrics

def display_report_analysis(metrics):
    """Display report analysis in a readable format"""
    display(Markdown("""
    ### Report Analysis
    
    #### Overall Metrics
    - Total Length: {} characters
    - Number of Sections: {}
    
    #### Section Breakdown
    """.format(metrics["total_length"], metrics["section_count"])))
    
    for section, data in metrics["sections"].items():
        display(Markdown(f"""
        **{section}**
        - Length: {data['length']} characters
        - Paragraphs: {data['paragraphs']}
        - Bullet Points: {data['bullet_points']}
        """))

In [ ]:
# Analyze our generated report
report_metrics = analyze_report_quality(report_content)
display_report_analysis(report_metrics)

# Let's also create an improved prompt based on this analysis
def create_enhanced_report_prompt(topic, metrics):
    """
    Create an improved prompt based on report analysis
    """
    # Calculate average section length for balance
    avg_section_length = metrics["total_length"] / max(metrics["section_count"], 1)
    
    enhanced_prompt = f"""
    Please create a comprehensive business report on:
    
    {topic}
    
    Requirements for a well-balanced report:
    1. Each major section should be approximately {int(avg_section_length/1.5)}-{int(avg_section_length*1.5)} characters
    2. Include 2-3 paragraphs per section for depth
    3. Use bullet points for key findings and recommendations
    4. Support claims with specific examples or data
    5. Include clear, actionable next steps
    
    Format using markdown with:
    - Clear section headers (# for main sections, ## for subsections)
    - Bullet points for lists
    - Bold text for key points
    - Tables for data presentation where appropriate
    """
    
    return enhanced_prompt

### Improving Report Generation

Our analysis shows several ways we can enhance our report generation:

1. **Better Section Balance**: Ensure sections have similar depth
2. **More Supporting Evidence**: Include specific data points and examples
3. **Enhanced Structure**: Use subsections and formatting for clarity
4. **Actionable Details**: Make recommendations more specific

Let's generate an improved version of our report using these insights.

In [ ]:
# Create an enhanced prompt using our insights
enhanced_prompt = create_enhanced_report_prompt(report_topic, report_metrics)

# Generate an improved report with a larger reasoning budget
improved_report_content, improved_response = generate_report(
    enhanced_prompt,
    reasoning_budget=8192,  # Increased for more thorough analysis
    max_tokens=3000       # Increased for more detailed content
)

# Analyze the improved report
improved_metrics = analyze_report_quality(improved_report_content)

# Compare the original and improved reports
print("Comparison of Original vs. Improved Report:")
print(f"Original Report Sections: {report_metrics['section_count']}")
print(f"Original Report Length: {report_metrics['total_length']} characters")
print(f"Improved Report Sections: {improved_metrics['section_count']}")
print(f"Improved Report Length: {improved_metrics['total_length']} characters\n")

# Display the improved report
display(Markdown("### Improved Report"))
display(Markdown(improved_report_content))

## Check-in Point: Report Generation

Before moving on to other content types, let's confirm our understanding of effective report generation:

**Key Insights:**
1. **Structure Matters**: Well-organized sections with balanced content
2. **Evidence-Based**: Supporting claims with data and examples
3. **Progressive Detail**: Moving from high-level summary to specific recommendations
4. **Format for Readability**: Using markdown formatting effectively

**Questions to Consider:**
- How did the increased reasoning budget affect report quality?
- Which improvements had the most impact on report usefulness?
- What other types of business reports could benefit from this approach?

Now let's move on to our next content type: creative writing.

## 2. Creative Writing and Storytelling

Creative writing presents different challenges than business reports. We need to focus on:
- Character development
- Plot structure
- Descriptive language
- Consistent narrative voice

Let's create a framework for generating creative content that maintains quality and coherence over longer pieces.

In [ ]:
def generate_creative_content(
    prompt,
    content_type="story",  # story, script, or poem
    reasoning_budget=6144,
    max_tokens=2500
):
    """
    Generate creative content using Claude's extended thinking
    
    Args:
        prompt (str): The creative writing prompt
        content_type (str): Type of creative content to generate
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        tuple: (content, response object)
    """
    # Create a structured prompt for creative writing
    creative_prompt = f"""
    Please create a piece of {content_type} based on the following prompt:
    
    {prompt}
    
    Before writing, create a detailed outline including:
    1. Characters/Elements
    2. Plot points/Structure
    3. Key themes
    4. Important descriptions
    
    Then develop the {content_type} with attention to:
    - Consistent narrative voice
    - Vivid descriptive language
    - Natural dialogue (if applicable)
    - Pacing and flow
    
    Use markdown formatting for structure and emphasis.
    """
    
    # Invoke Claude with extended thinking
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        creative_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=max_tokens
    )
    
    return claude_utils.extract_response_content(response), response

### Testing Creative Writing Generation

Let's test our creative writing function with a short story prompt. This will demonstrate how Claude can:
- Develop characters and plot
- Maintain narrative consistency
- Create engaging descriptions
- Balance showing and telling

We'll start with a prompt that provides some initial elements but leaves room for creativity.

In [ ]:
# Define our story prompt
story_prompt = """
Create a short story about a scientist who discovers an unusual pattern in their data 
that suggests something impossible is happening in their lab late at night.

The story should:
- Build suspense gradually
- Include both scientific and personal elements
- Have a surprising but satisfying resolution
- Be between 1000-1500 words

Main character traits:
- Dedicated researcher
- Initially skeptical
- Personally invested in their work
"""

# Generate the story
story_content, story_response = generate_creative_content(
    story_prompt,
    content_type="story",
    reasoning_budget=8192,  # Larger budget for creative development
    max_tokens=3000        # Allow for longer story
)

# Display generation metrics
print("Story Generation Metrics:")
print(f"Time taken: {story_response.get('_elapsed_time', 0):.2f} seconds")
print(f"Input tokens: {story_response.get('usage', {}).get('inputTokens', 0)}")
print(f"Output tokens: {story_response.get('usage', {}).get('outputTokens', 0)}\n")

# Display the story
display(Markdown("## Generated Story"))
display(Markdown(story_content))

### Analyzing Creative Writing Quality

Unlike business reports, creative writing requires different evaluation criteria. Let's analyze:
1. **Narrative Structure**: How well the story flows and builds
2. **Character Development**: How well characters are portrayed
3. **Language Usage**: Effectiveness of descriptions and dialogue
4. **Thematic Elements**: How well themes are developed

In [ ]:
def display_creative_analysis(metrics):
    """Display creative writing analysis in a readable format"""
    display(Markdown(f"""
    ### Creative Writing Analysis
    
    #### Structure Metrics
    - Total Words: {metrics['total_words']}
    - Paragraphs: {metrics['paragraphs']}
    - Scene Breaks: {metrics['scene_breaks']}
    
    #### Content Balance
    - Dialogue Lines: {metrics['dialogue_lines']}
    - Dialogue Percentage: {metrics['dialogue_percentage']:.1f}%
    - Descriptive Paragraphs: {metrics['descriptive_paragraphs']}
    
    #### Recommendations
    - Ideal dialogue percentage is typically 25-40% for engaging stories
    - Aim for a descriptive paragraph every 2-3 paragraphs
    - Consider adding more scene breaks if the story spans multiple scenes
    """))

In [ ]:
# Analyze our generated story
story_metrics = claude_utils.analyze_creative_writing(story_content)
display_creative_analysis(story_metrics)

# Let's create an enhanced creative prompt based on this analysis
def create_enhanced_creative_prompt(prompt, metrics):
    """
    Create an improved creative writing prompt based on analysis
    """
    current_dialogue_pct = metrics["dialogue_percentage"]
    dialogue_guidance = ("increase dialogue" if current_dialogue_pct < 25 
                        else "reduce dialogue" if current_dialogue_pct > 40 
                        else "maintain current dialogue level")
    
    enhanced_prompt = f"""
    Please create a story based on:
    
    {prompt}
    
    Writing Guidelines:
    1. Structure:
       - Aim for {metrics['total_words'] // 5}-{metrics['total_words'] // 3} words per scene
       - Include clear scene transitions
       - Balance action and reflection
    
    2. Character Development:
       - Show character traits through actions and dialogue
       - Include internal thoughts and reactions
       - Develop character arc through the story
    
    3. Dialogue and Description:
       - {dialogue_guidance} to achieve 25-40% dialogue
       - Include a descriptive paragraph every 2-3 paragraphs
       - Use all five senses in descriptions
    
    4. Pacing:
       - Start with a hook
       - Build tension gradually
       - Include 2-3 turning points
       - Provide satisfying resolution
    
    Format using markdown with scene breaks (***) between major sections.
    """
    
    return enhanced_prompt

In [ ]:
# Create enhanced prompt using our analysis
enhanced_story_prompt = create_enhanced_creative_prompt(story_prompt, story_metrics)

# Generate improved story with larger reasoning budget
improved_story_content, improved_story_response = generate_creative_content(
    enhanced_story_prompt,
    content_type="story",
    reasoning_budget=10240,  # Increased for more thorough development
    max_tokens=4000        # Increased for more detailed story
)

# Analyze the improved story
improved_story_metrics = claude_utils.analyze_creative_writing(improved_story_content)

# Compare original and improved stories
print("Comparison of Original vs. Improved Story:")
print(f"Original Story Words: {story_metrics['total_words']}")
print(f"Original Dialogue %: {story_metrics['dialogue_percentage']:.1f}%")
print(f"Improved Story Words: {improved_story_metrics['total_words']}")
print(f"Improved Dialogue %: {improved_story_metrics['dialogue_percentage']:.1f}%\n")

# Display the improved story
display(Markdown("## Improved Story"))
display(Markdown(improved_story_content))

### Advanced Creative Writing Techniques

Now that we've seen how to improve basic story generation, let's explore some advanced techniques:

1. **Multiple Viewpoints**: Generating stories with different character perspectives
2. **Complex Plot Structures**: Creating non-linear narratives
3. **World Building**: Developing detailed settings and environments
4. **Character Arcs**: Managing character development over longer pieces

Let's implement these advanced features in our creative writing generator.

In [ ]:
def generate_advanced_story(
    prompt,
    viewpoints=None,      # List of character viewpoints
    plot_structure="linear",  # linear, nonlinear, parallel
    world_building_depth="medium",  # light, medium, detailed
    reasoning_budget=12288,
    max_tokens=5000
):
    """
    Generate a story with advanced creative writing features
    
    Args:
        prompt (str): Base story prompt
        viewpoints (list): Character viewpoints to include
        plot_structure (str): Type of plot structure
        world_building_depth (str): Level of setting detail
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        tuple: (content, response object)
    """
    # Create structured prompt for advanced story
    advanced_prompt = f"""
    Create an advanced story incorporating these elements:
    
    Base Prompt:
    {prompt}
    
    Technical Requirements:
    1. Plot Structure: {plot_structure}
    2. World Building Depth: {world_building_depth}
    """
    
    # Add viewpoint requirements if specified
    if viewpoints:
        advanced_prompt += f"\n3. Viewpoint Characters:\n"
        for char in viewpoints:
            advanced_prompt += f"   - {char}\n"
    
    advanced_prompt += """
    Development Guidelines:
    - Create distinct voice for each viewpoint
    - Maintain consistent world details
    - Weave plot threads together naturally
    - Balance exposition and action
    - Use scene breaks (***) between viewpoints
    """
    
    # Generate the story
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        advanced_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=max_tokens
    )
    
    return claude_utils.extract_response_content(response), response

### Testing Advanced Story Generation

Let's create a more complex story that demonstrates:
- Multiple character viewpoints
- Non-linear plot structure
- Detailed world-building

We'll use a science fiction premise that allows us to explore these elements effectively.

In [ ]:
# Define our advanced story parameters
advanced_story_prompt = """
Create a story about the first contact between humanity and an alien civilization, 
told from multiple perspectives as strange objects appear simultaneously in different parts of the world.

Setting elements:
- Near-future Earth (2045)
- Advanced AI monitoring systems
- Quantum communication technology
- Global climate challenges

Key events should be shown from different viewpoints to build a complete picture of what's happening.
"""

# Define viewpoint characters
viewpoints = [
    "Dr. Sarah Chen - Quantum physicist at CERN",
    "Captain Maya Rodriguez - Space Force orbital defense",
    "AI System GAIA-9 - Global monitoring network",
    "Diplomat Kwame Osei - UN First Contact Protocol leader"
]

# Generate advanced story
complex_story_content, complex_story_response = generate_advanced_story(
    prompt=advanced_story_prompt,
    viewpoints=viewpoints,
    plot_structure="nonlinear",
    world_building_depth="detailed",
    reasoning_budget=16384,  # Large budget for complex narrative
    max_tokens=6000         # Extended length for multiple viewpoints
)

# Analyze the complex story
complex_story_metrics = claude_utils.analyze_creative_writing(complex_story_content)

# Display metrics
print("Advanced Story Generation Metrics:")
print(f"Time taken: {complex_story_response.get('_elapsed_time', 0):.2f} seconds")
print(f"Total words: {complex_story_metrics['total_words']}")
print(f"Scene breaks: {complex_story_metrics['scene_breaks']}")
print(f"Dialogue percentage: {complex_story_metrics['dialogue_percentage']:.1f}%\n")

# Display the story
display(Markdown("## Advanced Multi-Viewpoint Story"))
display(Markdown(complex_story_content))

### Check-in Point: Advanced Creative Writing

Let's review what we've learned about generating complex creative content:

**Key Techniques:**
1. **Viewpoint Management**: How to maintain distinct character voices
2. **Plot Complexity**: Handling non-linear narratives effectively
3. **World Building**: Creating consistent, detailed settings
4. **Story Balance**: Managing pacing across multiple threads

**Questions to Consider:**
- How did the increased reasoning budget affect narrative complexity?
- Which viewpoint transitions worked best?
- How well did the world-building details maintain consistency?
- What other story types could benefit from these techniques?

Next, let's move on to our final content type: technical documentation.

## 3. Technical Documentation Generation

Technical documentation requires a different approach than reports or creative writing. Key elements include:
- Clear structure and organization
- Precise technical language
- Code examples and explanations
- Step-by-step instructions

Let's create a framework for generating comprehensive technical documentation that maintains accuracy and clarity.

In [ ]:
def generate_technical_doc(
    prompt,
    doc_type="guide",  # guide, api_doc, tutorial
    include_code=True,
    technical_level="intermediate",  # beginner, intermediate, advanced
    reasoning_budget=8192,
    max_tokens=4000
):
    """
    Generate technical documentation using Claude's extended thinking
    
    Args:
        prompt (str): The documentation requirements
        doc_type (str): Type of technical document
        include_code (bool): Whether to include code examples
        technical_level (str): Target expertise level
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        tuple: (content, response object)
    """
    # Create structured prompt for technical documentation
    tech_prompt = f"""
    Please create {doc_type} technical documentation for the following:
    
    {prompt}
    
    Target audience: {technical_level} level
    
    Documentation structure:
    1. Overview
    2. Prerequisites
    3. Technical Background
    4. Implementation Details
    5. Usage Examples
    6. Best Practices
    7. Troubleshooting
    
    Requirements:
    - Use clear, precise technical language
    - Include appropriate code examples
    - Provide step-by-step instructions where needed
    - Add notes and warnings for important considerations
    - Use markdown formatting for readability
    
    Format code examples with appropriate language tags for syntax highlighting.
    """
    
    # Invoke Claude with extended thinking
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        tech_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=max_tokens
    )
    
    return claude_utils.extract_response_content(response), response

### Testing Technical Documentation Generation

Let's test our documentation generator with a practical example: creating documentation for a REST API endpoint. This will demonstrate how Claude can:
- Explain technical concepts clearly
- Provide accurate code examples
- Structure information logically
- Address common use cases and issues

In [ ]:
# Define our API documentation prompt
api_doc_prompt = """
Create documentation for a user authentication API endpoint with the following specifications:

Endpoint: /api/v1/auth/login
Method: POST
Purpose: Authenticate users and return JWT token

Technical Details:
- Rate limiting: 5 requests per minute
- Token expiration: 24 hours
- Required fields: email, password
- Optional fields: remember_me
- Supports both JSON and form-data

Include:
- Request/response formats
- Authentication flows
- Error handling
- Security considerations
- Code examples in Python and JavaScript
"""

# Generate the API documentation
api_doc_content, api_doc_response = generate_technical_doc(
    prompt=api_doc_prompt,
    doc_type="api_doc",
    technical_level="intermediate",
    reasoning_budget=8192,
    max_tokens=4000
)

# Display generation metrics
print("Technical Documentation Generation Metrics:")
print(f"Time taken: {api_doc_response.get('_elapsed_time', 0):.2f} seconds")
print(f"Input tokens: {api_doc_response.get('usage', {}).get('inputTokens', 0)}")
print(f"Output tokens: {api_doc_response.get('usage', {}).get('outputTokens', 0)}\n")

# Display the documentation
display(Markdown("## API Documentation"))
display(Markdown(api_doc_content))

In [ ]:
def display_technical_analysis(metrics):
    """Display technical documentation analysis"""
    display(Markdown(f"""
    ### Technical Documentation Analysis
    
    #### Content Structure
    - Total Words: {metrics['total_words']}
    - Main Sections: {metrics['sections']}
    - Subsections: {metrics['subsections']}
    
    #### Technical Elements
    - Code Examples: {metrics['code_blocks']}
    - Usage Examples: {metrics['examples']}
    - Warnings/Notes: {metrics['warnings']}
    - References/Links: {metrics['links']}
    
    #### Recommendations
    - Aim for 2-3 code examples per major concept
    - Include warning notes for common pitfalls
    - Add cross-references where appropriate
    """))

In [ ]:
# Analyze our generated API documentation
tech_doc_metrics = claude_utils.analyze_technical_doc(api_doc_content)
display_technical_analysis(tech_doc_metrics)

# Create an enhanced prompt based on this analysis
def create_enhanced_tech_doc_prompt(prompt, metrics):
    """
    Create an improved technical documentation prompt based on analysis
    """
    enhanced_prompt = f"""
    Please create detailed technical documentation for:
    
    {prompt}
    
    Documentation Requirements:
    1. Structure:
       - Include {max(metrics['sections'] + 2, 7)} main sections
       - Create {max(metrics['subsections'] + 3, 10)} subsections for detailed topics
       - Add a table of contents at the start
    
    2. Technical Content:
       - Provide at least {max(metrics['code_blocks'] + 2, 5)} code examples
       - Include {max(metrics['examples'] + 2, 4)} usage examples
       - Add {max(metrics['warnings'] + 2, 5)} warning/note boxes for important considerations
    
    3. Clarity and Depth:
       - Start each section with a brief overview
       - Include practical examples after each concept
       - Provide troubleshooting guidance
       - Add security best practices
    
    4. Formatting:
       - Use markdown headers for structure (# for main, ## for sub)
       - Format code blocks with language-specific syntax highlighting
       - Use tables for parameter listings
       - Add blockquotes for important notes
    
    The documentation should be thorough yet practical, suitable for developers implementing the API.
    """
    
    return enhanced_prompt

In [ ]:
# Generate improved API documentation
enhanced_api_prompt = create_enhanced_tech_doc_prompt(api_doc_prompt, tech_doc_metrics)

improved_api_doc_content, improved_api_doc_response = generate_technical_doc(
    prompt=enhanced_api_prompt,
    doc_type="api_doc",
    technical_level="intermediate",
    reasoning_budget=12288,  # Increased for more thorough documentation
    max_tokens=5000         # Increased for more detailed content
)

# Analyze the improved documentation
improved_tech_doc_metrics = claude_utils.analyze_technical_doc(improved_api_doc_content)

# Compare original and improved documentation
print("Comparison of Original vs. Improved Documentation:")
print(f"Original Words: {tech_doc_metrics['total_words']}")
print(f"Original Code Examples: {tech_doc_metrics['code_blocks']}")
print(f"Improved Words: {improved_tech_doc_metrics['total_words']}")
print(f"Improved Code Examples: {improved_tech_doc_metrics['code_blocks']}\n")

# Display the improved documentation
display(Markdown("## Improved API Documentation"))
display(Markdown(improved_api_doc_content))

## Check-in Point: Technical Documentation

Let's review what we've learned about generating effective technical documentation:

**Key Elements:**
1. **Structure**: Clear organization with appropriate sections and subsections
2. **Code Examples**: Practical, well-commented implementation examples
3. **Usage Guidance**: Clear instructions and best practices
4. **Error Handling**: Comprehensive troubleshooting information

**Questions to Consider:**
- How did the enhanced structure improve documentation clarity?
- Which aspects of the documentation would be most helpful to developers?
- What other types of technical documentation could benefit from this approach?
- How could we further improve code example generation?

Now let's move to our final section on managing and validating long-form outputs.

## 4. Managing and Validating Long-Form Outputs

When generating extensive content, it's crucial to:
- Ensure consistency throughout the document
- Validate technical accuracy
- Check for completeness
- Maintain structural integrity

Let's create tools to help manage and validate long-form content generation.

In [ ]:
def display_validation_results(validation):
    """Display content validation results in a readable format"""
    status = "✅ PASSED" if validation["pass"] else "❌ FAILED"
    
    display(Markdown(f"""
    ### Content Validation Results: {status}
    
    #### Document Structure
    - Total Length: {validation['total_length']} characters
    - Number of Sections: {validation['section_count']}
    
    #### Section Analysis
    """))
    
    for section, data in validation["sections"].items():
        status = "✅" if data["meets_min_length"] else "❌"
        display(Markdown(f"""
        **{section}** {status}
        - Length: {data['length']} characters
        - Paragraphs: {data['paragraphs']}
        """))
    
    if validation["issues"]:
        display(Markdown("\n#### Issues Found ❌"))
        for issue in validation["issues"]:
            display(Markdown(f"- {issue}"))
    
    if validation["warnings"]:
        display(Markdown("\n#### Warnings ⚠️"))
        for warning in validation["warnings"]:
            display(Markdown(f"- {warning}"))

In [ ]:
# Let's validate our different types of content

# 1. Validate the business report
report_validation = claude_utils.validate_long_form_content(
    report_content,
    content_type="report",
    required_sections=[
        "Executive Summary",
        "Background and Context",
        "Analysis and Findings",
        "Recommendations",
        "Implementation Plan",
        "Conclusion"
    ],
    min_section_length=200
)

print("Business Report Validation:")
display_validation_results(report_validation)

# 2. Validate the creative story
story_validation = claude_utils.validate_long_form_content(
    complex_story_content,
    content_type="creative",
    min_section_length=300  # Longer sections for creative writing
)

print("\nCreative Story Validation:")
display_validation_results(story_validation)

# 3. Validate the technical documentation
tech_doc_validation = claude_utils.validate_long_form_content(
    improved_api_doc_content,
    content_type="technical",
    required_sections=[
        "Overview",
        "Prerequisites",
        "Technical Background",
        "Implementation Details",
        "Usage Examples",
        "Best Practices",
        "Troubleshooting"
    ],
    min_section_length=150
)

print("\nTechnical Documentation Validation:")
display_validation_results(tech_doc_validation)

### Improving Content Based on Validation

Now that we've identified potential issues in our content, let's create a function that can help address them by:
1. Expanding short sections
2. Adding missing required content
3. Balancing content types (dialogue, code examples, etc.)
4. Maintaining consistency across sections

In [ ]:
def improve_content_based_on_validation(content, validation_results, content_type="general"):
    """
    Create a prompt to improve content based on validation results
    
    Args:
        content (str): Original content
        validation_results (dict): Results from content validation
        content_type (str): Type of content being improved
        
    Returns:
        str: Enhanced prompt for content improvement
    """
    improvement_prompt = f"""
    Please improve the following content, addressing these specific issues:
    
    Original Content:
    {content}
    
    Required Improvements:
    """
    
    # Add specific improvements based on validation issues
    if validation_results["issues"]:
        improvement_prompt += "\nCritical Issues to Fix:\n"
        for issue in validation_results["issues"]:
            improvement_prompt += f"- {issue}\n"
    
    if validation_results["warnings"]:
        improvement_prompt += "\nWarnings to Address:\n"
        for warning in validation_results["warnings"]:
            improvement_prompt += f"- {warning}\n"
    
    # Add content-type specific guidance
    if content_type == "technical":
        improvement_prompt += """
        Technical Documentation Requirements:
        - Ensure each section is comprehensive and clearly explained
        - Add or expand code examples where missing
        - Include error handling and edge cases
        - Add more practical usage examples
        """
    elif content_type == "creative":
        improvement_prompt += """
        Creative Writing Requirements:
        - Balance dialogue and narrative
        - Enhance descriptive passages
        - Strengthen character development
        - Improve scene transitions
        """
    elif content_type == "report":
        improvement_prompt += """
        Report Requirements:
        - Strengthen data analysis
        - Make recommendations more specific
        - Add implementation details
        - Enhance executive summary
        """
    
    improvement_prompt += """
    
    Please maintain the original structure while expanding and improving the content.
    Ensure consistency across all sections and maintain the document's tone and style.
    """
    
    return improvement_prompt

In [ ]:
# Create improvement prompt for technical documentation
tech_improvement_prompt = improve_content_based_on_validation(
    improved_api_doc_content,
    tech_doc_validation,
    content_type="technical"
)

# Generate improved version with higher reasoning budget
final_tech_doc_content, final_tech_response = generate_technical_doc(
    prompt=tech_improvement_prompt,
    doc_type="api_doc",
    technical_level="intermediate",
    reasoning_budget=16384,  # Increased for more thorough improvements
    max_tokens=6000         # Increased to allow for expanded content
)

# Validate the final version
final_tech_validation = claude_utils.validate_long_form_content(
    final_tech_doc_content,
    content_type="technical",
    required_sections=[
        "Overview",
        "Prerequisites",
        "Technical Background",
        "Implementation Details",
        "Usage Examples",
        "Best Practices",
        "Troubleshooting"
    ],
    min_section_length=150
)

# Compare versions
print("Documentation Version Comparison:")
print(f"Original Length: {len(improved_api_doc_content)} characters")
print(f"Final Length: {len(final_tech_doc_content)} characters")
print(f"Original Issues: {len(tech_doc_validation['issues'])}")
print(f"Final Issues: {len(final_tech_validation['issues'])}\n")

# Display final validation results
display(Markdown("### Final Documentation Validation"))
display_validation_results(final_tech_validation)

# Display the final improved documentation
display(Markdown("## Final Improved API Documentation"))
display(Markdown(final_tech_doc_content))

## Managing Long-Form Content Generation: Best Practices

Based on our experiments with different content types and validation approaches, here are key best practices for generating and managing long-form content:

1. **Pre-generation Planning**
   - Define clear section requirements
   - Set appropriate length guidelines
   - Specify content-type specific elements
   - Configure reasonable reasoning budgets

2. **Content Generation**
   - Use structured prompts
   - Enable sufficient reasoning budget
   - Include validation requirements
   - Request specific examples/elements

3. **Validation and Improvement**
   - Check for structural completeness
   - Validate content-specific elements
   - Address issues systematically
   - Maintain consistency in improvements

4. **Quality Control**
   - Regular validation checks
   - Progressive improvements
   - Content-type specific metrics
   - Consistency verification

## Final Check-in Point: Long-Form Content Generation

Let's review the key concepts we've covered in this lesson:

**1. Content Type Strategies**
- Business Reports: Structure, analysis, and recommendations
- Creative Writing: Character development, plot, and narrative flow
- Technical Documentation: Clarity, accuracy, and practical examples

**2. Extended Thinking Application**
- Using larger reasoning budgets for complex content
- Structuring prompts for comprehensive outputs
- Managing content consistency across longer pieces

**3. Validation and Improvement**
- Automated content validation
- Issue identification and resolution
- Content-specific quality metrics
- Systematic improvement processes

**Questions to Consider:**
- Which content type benefited most from extended thinking?
- How did validation help improve content quality?
- What other types of long-form content could use these techniques?
- How would you adapt these approaches for your specific needs?

## Conclusion and Next Steps

### Key Takeaways
1. **Extended Thinking Power**: Claude 3.7's extended thinking capability enables generation of high-quality, consistent long-form content
2. **Content-Type Optimization**: Different content types require different approaches and validation strategies
3. **Quality Control**: Systematic validation and improvement processes are crucial for long-form content
4. **Iterative Improvement**: Progressive refinement leads to better quality outputs

### Practical Applications
- Business report generation
- Creative writing and storytelling
- Technical documentation
- Educational content
- Research papers
- Product descriptions
- Policy documents

### Next Steps
1. **Experiment** with different content types
2. **Customize** validation criteria for your needs
3. **Develop** specialized prompts for your use cases
4. **Build** automated workflows for content generation

### Additional Resources
- Claude 3.7 documentation
- Content generation best practices
- Extended thinking optimization guides
- Validation framework documentation

## Practice Exercises

To reinforce what you've learned, try these exercises:

1. **Business Report Generation**
   - Create a market analysis report
   - Generate a project proposal
   - Develop a strategic plan

2. **Creative Writing**
   - Write a short story with multiple viewpoints
   - Create a detailed scene description
   - Develop a character background

3. **Technical Documentation**
   - Document a complex API
   - Create a user guide
   - Write installation instructions

4. **Content Validation**
   - Develop custom validation criteria
   - Create specialized improvement prompts
   - Build automated validation workflows

Remember to use the techniques, validation tools, and best practices covered in this lesson when working on these exercises.